# Model Predicitons

## Set up environment

In [15]:
sys.path.insert(0, './model-training/functions')

In [17]:
import sys
import pandas as pd
import sqlite3
import pathlib
import joblib

sys.path.append("functions") 
import prediction_functions as pf

sys.path.insert(0, '../model-training/functions')
import training_config as tc

## Get upcoming match data

In [18]:
# Get to the root directory
project_root = pathlib.Path().absolute().parent.parent

# Now construct the relative path to your SQLite database
db_path = project_root / "data" / "footy-tipper-db.sqlite"

# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file
with open('sql/inference_data.sql', 'r') as file:
    query = file.read()

inference_data = pd.read_sql_query(query, con)

# Don't forget to close the connection
con.close()

inference_data

,game_id,round_id,round_name,game_number,game_state_name,start_time,start_time_utc,venue_name,city,crowd,...,matchup_form,state_of_origin,home_elo,away_elo,elo_diff,home_elo_prob,away_elo_prob,elo_draw_prob,elo_prob_diff,home_ground_advantage
0,2.023111e+10,21.0,Round 21,1.0,Pre Game,1.689883e+09,1.689847e+09,WIN Stadium,Wollongong,None,...,-3.0,0.0,1461.378294,1445.906775,15.471520,0.505018,0.455247,0.039735,0.049772,2.181933
1,2.023111e+10,21.0,Round 21,2.0,Pre Game,1.689970e+09,1.689926e+09,Go Media Stadium,Auckland,None,...,-1.0,0.0,1542.588164,1489.489972,53.098192,0.568196,0.412449,0.019355,0.155746,3.869933
2,2.023111e+10,21.0,Round 21,3.0,Pre Game,1.689970e+09,1.689934e+09,Sunshine Coast Stadium,Sunshine Coast,None,...,1.0,0.0,1510.479292,1512.441039,-1.961747,0.491040,0.489373,0.019587,0.001667,3.719133
3,2.023111e+10,21.0,Round 21,4.0,Pre Game,1.690038e+09,1.690002e+09,Cbus Super Stadium,Gold Coast,None,...,-3.0,0.0,1489.984523,1479.009827,10.974695,0.509283,0.471130,0.019587,0.038153,0.235867
4,2.023111e+10,21.0,Round 21,5.0,Pre Game,1.690047e+09,1.690011e+09,McDonald Jones Stadium,Newcastle,None,...,-5.0,0.0,1514.952325,1528.123710,-13.171385,0.475229,0.505184,0.019587,-0.029955,-4.172500
5,2.023111e+10,21.0,Round 21,6.0,Pre Game,1.690054e+09,1.690018e+09,Queensland Country Bank Stadium,Townsville,None,...,1.0,0.0,1541.048520,1521.494052,19.554468,0.510642,0.449623,0.039735,0.061019,0.804033
6,2.023111e+10,21.0,Round 21,7.0,Pre Game,1.690121e+09,1.690085e+09,BlueBet Stadium,Penrith,None,...,5.0,0.0,1545.681836,1435.667133,110.014703,0.633109,0.331177,0.035714,0.301932,18.747767
7,2.023111e+10,21.0,Round 21,8.0,Pre Game,1.690128e+09,1.690092e+09,PointsBet Stadium,Sydney,None,...,-3.0,0.0,1514.378692,1490.119806,24.258887,0.517111,0.443154,0.039735,0.073957,7.649700


## Load the Footy Tipper model

In [19]:
footy_tipper = joblib.load(project_root / "models" / "footy_tipper.pkl")
label_encoder = joblib.load(project_root / "models" / "label_encoder.pkl")

In [22]:
footy_tipper

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.4314917348568336, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.1730410622323778, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.08178239522486815, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=27, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

# Make predictions

In [21]:
# You'll need to use the 'optimal_features' variable which was returned from 'train_model_pipeline' or 'train_and_select_best_model' function
optimal_features = footy_tipper.named_steps['columntransformer'].transformers_[1][1].named_steps['onehotencoder'].get_feature_names(input_features=categorical_columns)

X_inference = pf.preprocess_inference_data(inference_data, tc.predictors, label_encoder, optimal_features)

# Make predictions
predictions = footy_tipper.predict(X_inference)

AttributeError: 'XGBClassifier' object has no attribute 'named_steps'

## Write predictions to the database
With this final step, the comprehensive procedure concludes. The sophisticated model, once trained, has offered its predictions for the current week's NRL matches. The generated predictions have been appropriately stored within the database, readily available for ensuing analysis and application. The rigorous pipeline, hence, has successfully accomplished its mission, ensuring the implementation of robust model training, precise generation of predictions, and secure archival of data.

In [ ]:
# Connect to the SQLite database
con = sqlite3.connect(str(db_path))

# Read SQL query from external SQL file and create table
with open('sql/create_table.sql', 'r') as file:
    create_table_query = file.read()
con.execute(create_table_query)

# Read SQL query from external SQL file for insertion
with open('sql/insert_into_table.sql', 'r') as file:
    insert_into_table_query = file.read()

# Write each row in the DataFrame to the database
for index, row in predictions_df.iterrows():
    con.execute(insert_into_table_query, (
        row['game_id'], 
        row['home_team_result'],
        row['home_team_win_prob'],
        row['home_team_lose_prob']
    ))

# Commit the transaction
con.commit()

# Close the connection
con.close()